In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost.sklearn import XGBRegressor 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [2]:
train = pd.read_csv('training.csv')
test = pd.read_csv('test_features.csv')

In [3]:
train.shape

(1000000, 9)

In [5]:
clean_df = train[train.salary > 0]
clean_df = clean_df.drop('jobId', axis = 1)
clean_df.shape

(999995, 8)

Building Data Frames for the models

In [6]:
feature_df = clean_df.drop('salary',axis = 1)
target = clean_df['salary']
job_id = test['jobId']
test = test.drop('jobId', axis = 1)

#### Building a ColumnTransformer
One Hot Encoding the categorical variables and passing the numerical values through.

In [8]:
transformer = make_column_transformer(
(OneHotEncoder(), ['companyId','jobType',
            'degree','major','industry']),
            remainder = 'passthrough')

#### Baseline Model

In [20]:
average = np.mean(clean_df['salary'])
print(F"The average salary for the entire dataset is {average}")
y_true = list(target)
y_pred = average

The average salary for the entire dataset is 116.06239831199156


In [21]:
y_pred

116.06239831199156